In [36]:
import openai, os
from maxim.logger.components.generation import GenerationConfig, GenerationError
from maxim.logger.components.retrieval import RetrievalConfig
from maxim.logger.components.trace import Trace, TraceConfig
from maxim.logger.components.session import Session, SessionConfig
from maxim.logger.components.span import Span, SpanConfig
from maxim.logger.logger import Logger, LoggerConfig
from uuid import uuid4
from dotenv import load_dotenv
import os
from time import sleep, time
import tiktoken
load_dotenv()


True

In [37]:
base_url = "https://app.getmaxim.ai"
maxim_api_key = os.environ["MAXIM_API_KEY"]
log_repository_id = os.environ["LOG_REPO_ID"]
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Set up logger
logger_config = LoggerConfig(id=log_repository_id)
logger = Logger(config=logger_config, api_key=maxim_api_key, base_url=base_url)

# Set up session
sessionId = str(uuid4())
session_config = SessionConfig(id=sessionId)
session = logger.session(session_config)

# Set up trace
traceId = str(uuid4())
trace_config = TraceConfig(id=traceId)
trace = session.trace(trace_config)

In [38]:
import pandas as pd
from swarm import Agent

def load_logistics_data():
    data = {
        "Shipment ID": ["S001", "S002", "S003"],
        "Status": ["In Transit", "Delivered", "Processing"],
        "Origin": ["New York", "Los Angeles", "Chicago"],
        "Destination": ["Miami", "Seattle", "Houston"],
        "Estimated Delivery": ["2024-11-08", "2024-11-07", "2024-11-09"],
        "Current Location": ["Atlanta", "Seattle", "Dallas"],
        "Carrier": ["FedEx", "UPS", "DHL"]
    }
    return pd.DataFrame(data)

def load_warehouse_data():
    data = {
        "Warehouse ID": ["W001", "W002", "W003"],
        "Location": ["New York", "Los Angeles", "Chicago"],
        "Capacity": [1000, 1500, 800],
        "Current Occupancy": [750, 1200, 600],
        "Temperature": ["Room", "Cold", "Room"],
        "Special Features": ["High Security", "Refrigerated", "Automated"]
    }
    return pd.DataFrame(data)

def load_inventory_data():
    data = {
        "Product ID": ["P001", "P002", "P003"],
        "Name": ["Electronics", "Pharmaceuticals", "Furniture"],
        "Warehouse ID": ["W001", "W002", "W003"],
        "Quantity": [100, 250, 50],
        "Reorder Point": [20, 50, 10],
        "Storage Requirements": ["Standard", "Temperature Controlled", "Large Space"]
    }
    return pd.DataFrame(data)

class IntegratedLogisticsData:
    def __init__(self):
        self.logistics_data = load_logistics_data()
        self.warehouse_data = load_warehouse_data()
        self.inventory_data = load_inventory_data()

    def get_shipment_info(self, shipment_id):
        if shipment_id in self.logistics_data['Shipment ID'].values:
            return self.logistics_data[self.logistics_data['Shipment ID'] == shipment_id].to_dict('records')[0]
        return None

    def get_warehouse_info(self, warehouse_id):
        if warehouse_id in self.warehouse_data['Warehouse ID'].values:
            return self.warehouse_data[self.warehouse_data['Warehouse ID'] == warehouse_id].to_dict('records')[0]
        return None

    def get_inventory_info(self, product_id):
        if product_id in self.inventory_data['Product ID'].values:
            return self.inventory_data[self.inventory_data['Product ID'] == product_id].to_dict('records')[0]
        return None

integrated_logistics = IntegratedLogisticsData()

# Agent functions
def track_shipment(shipment_id):
    """Track the status and location of a specific shipment."""
    shipment_info = integrated_logistics.get_shipment_info(shipment_id)
    if shipment_info:
        return (f"Shipment {shipment_id} Status:\n"
                f"Current Status: {shipment_info['Status']}\n"
                f"Current Location: {shipment_info['Current Location']}\n"
                f"Estimated Delivery: {shipment_info['Estimated Delivery']}\n"
                f"Carrier: {shipment_info['Carrier']}")
    return f"No shipment found with ID {shipment_id}"

def check_warehouse_capacity(warehouse_id):
    """Check the current capacity and occupancy of a warehouse."""
    warehouse_info = integrated_logistics.get_warehouse_info(warehouse_id)
    if warehouse_info:
        occupancy_percentage = (warehouse_info['Current Occupancy'] / warehouse_info['Capacity']) * 100
        return (f"Warehouse {warehouse_id} Status:\n"
                f"Location: {warehouse_info['Location']}\n"
                f"Capacity: {warehouse_info['Capacity']} units\n"
                f"Current Occupancy: {warehouse_info['Current Occupancy']} units\n"
                f"Occupancy Rate: {occupancy_percentage:.1f}%\n"
                f"Special Features: {warehouse_info['Special Features']}")
    return f"No warehouse found with ID {warehouse_id}"

def check_inventory_levels(product_id):
    """Check current inventory levels and reorder points for a product."""
    inventory_info = integrated_logistics.get_inventory_info(product_id)
    if inventory_info:
        status = "Normal"
        if inventory_info['Quantity'] <= inventory_info['Reorder Point']:
            status = "Reorder Required"
        return (f"Product {product_id} Inventory Status:\n"
                f"Name: {inventory_info['Name']}\n"
                f"Current Quantity: {inventory_info['Quantity']}\n"
                f"Reorder Point: {inventory_info['Reorder Point']}\n"
                f"Status: {status}\n"
                f"Storage Requirements: {inventory_info['Storage Requirements']}")
    return f"No product found with ID {product_id}"

# Define agents
logistics_agent = Agent(
    name="Logistics Agent",
    instructions="Handle shipment tracking and delivery status queries. Provide detailed information about shipment location and estimated delivery times.",
    functions=[track_shipment]
)

warehouse_agent = Agent(
    name="Warehouse Agent",
    instructions="Monitor warehouse operations, capacity, and special storage requirements. Provide information about warehouse utilization and capabilities.",
    functions=[check_warehouse_capacity]
)

inventory_agent = Agent(
    name="Inventory Agent",
    instructions="Track inventory levels, monitor stock movements, and alert when reorder points are reached. Provide detailed inventory status reports.",
    functions=[check_inventory_levels]
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="Determine which specialized agent is best suited to handle the user's logistics-related request, and transfer the conversation accordingly.",
)

# Transfer functions
def transfer_to_logistics():
    return logistics_agent

def transfer_to_warehouse():
    return warehouse_agent

def transfer_to_inventory():
    return inventory_agent

def transfer_back_to_triage():
    print (triage_agent)
    return triage_agent

# Set up transfer functions for each agent
triage_agent.functions = [transfer_to_logistics, transfer_to_warehouse, transfer_to_inventory]
logistics_agent.functions.append(transfer_back_to_triage)
warehouse_agent.functions.append(transfer_back_to_triage)
inventory_agent.functions.append(transfer_back_to_triage)

In [39]:
from swarm import Swarm
client = Swarm()


In [40]:
query = "What is the shipment status for S001?"
response = client.run(
   agent=triage_agent,
   messages=[{"role":"user", "content": query }]
)
# print(response.messages[-1]["content"])
print(response)

messages=[{'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_s7iFpbnl0TP8kwSvnYaVSIJ1', 'function': {'arguments': '{}', 'name': 'transfer_to_logistics'}, 'type': 'function'}], 'sender': 'Triage Agent'}, {'role': 'tool', 'tool_call_id': 'call_s7iFpbnl0TP8kwSvnYaVSIJ1', 'tool_name': 'transfer_to_logistics', 'content': '{"assistant": "Logistics Agent"}'}, {'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_eg3Lg53GslUJMWXIDXoAeMqC', 'function': {'arguments': '{"shipment_id":"S001"}', 'name': 'track_shipment'}, 'type': 'function'}], 'sender': 'Logistics Agent'}, {'role': 'tool', 'tool_call_id': 'call_eg3Lg53GslUJMWXIDXoAeMqC', 'tool_name': 'track_shipment', 'content': 'Shipment S001 Status:\nCurrent Status: In Transit\nCurrent Location: Atlanta\nEstimated Delivery: 2024-11-08\nCarrier: FedEx'}, {'content': 'The shipment S001 is currently in transit. It is located in Atlanta, and t

In [41]:
span = None
for message in response.messages:
    if message["role"] == "assistant":
        print(message['sender'])
        if span != None:
            #ending the earlier span
            span.end()
        span = trace.span(SpanConfig(id=str(uuid4()), name=f"Agent {message['sender']}"))
        span.event(str(uuid4()), "Agent response", message)
        sleep(1)
    if message["role"] == "tool":
        if span != None:
            span.event(str(uuid4()), f"Tool called: {message['tool_name']}", message)
        sleep(1)
#closing the last span
span.end()
generationConfig = GenerationConfig(id=str(uuid4()), name="generation", provider="openai", model="gpt-4o", model_parameters={"temperature": 0}, messages=[{"role":"user", "content": query }])
generation = trace.generation(generationConfig)
enc = tiktoken.get_encoding("o200k_base")
llm_output = response.messages[-1]["content"]
print("on_llm_end")
messages = generationConfig.messages
messages_string = ''.join(["role: " + entry["role"] + " content: " + entry['content'] for entry in messages])
prompt_tokens = len(enc.encode(messages_string))
completion_tokens = len(enc.encode(llm_output))
generation.result({
            "id": generation.id,
            "object": "text_completion",
            "created": int(time()),
            "model": generationConfig.model,
            "choices": [
                {
                    "index": 0,
                    "text": response.messages[-1]["content"],
                    "logprobs": None,
                    "finish_reason": "stop",
                },
            ],
            "usage": {
                "prompt_tokens": prompt_tokens,
                "completion_tokens": completion_tokens,
                "total_tokens": prompt_tokens+completion_tokens,
            },
        })
generation.end()
trace.end()

Triage Agent
Logistics Agent
Logistics Agent
on_llm_end


In [42]:
## Example of a response : Helpful to understand the structure of the response

messages=[{'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_ljrwiFs5S70FXL8TvSt7rbYy', 'function': {'arguments': '{}', 'name': 'transfer_to_logistics'}, 'type': 'function'}], 'sender': 'Triage Agent'}, {'role': 'tool', 'tool_call_id': 'call_ljrwiFs5S70FXL8TvSt7rbYy', 'tool_name': 'transfer_to_logistics', 'content': '{"assistant": "Logistics Agent"}'}, {'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_aNDjzfrUuet6x5fw417ghlX8', 'function': {'arguments': '{"shipment_id":"S001"}', 'name': 'track_shipment'}, 'type': 'function'}], 'sender': 'Logistics Agent'}, {'role': 'tool', 'tool_call_id': 'call_aNDjzfrUuet6x5fw417ghlX8', 'tool_name': 'track_shipment', 'content': 'Shipment S001 Status:\nCurrent Status: In Transit\nCurrent Location: Atlanta\nEstimated Delivery: 2024-11-08\nCarrier: FedEx'}, {'content': 'The shipment with ID S001 is currently in transit. It is located in Atlanta, and the estimated delivery date is November 8, 2024. The carrier handling the shipment is FedEx.', 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': None, 'sender': 'Logistics Agent'}] 
# # agent=Agent(name='Logistics Agent', model='gpt-4o', instructions='Handle shipment tracking and delivery status queries. Provide detailed information about shipment location and estimated delivery times.', functions=[<function track_shipment at 0x117291e40>, <function transfer_back_to_triage at 0x117291300>], tool_choice=None, parallel_tool_calls=True) 
# # context_variables={}
print(messages[0], messages[1], messages[2], messages[3], messages[4],sep="\n")

{'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_ljrwiFs5S70FXL8TvSt7rbYy', 'function': {'arguments': '{}', 'name': 'transfer_to_logistics'}, 'type': 'function'}], 'sender': 'Triage Agent'}
{'role': 'tool', 'tool_call_id': 'call_ljrwiFs5S70FXL8TvSt7rbYy', 'tool_name': 'transfer_to_logistics', 'content': '{"assistant": "Logistics Agent"}'}
{'content': None, 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_aNDjzfrUuet6x5fw417ghlX8', 'function': {'arguments': '{"shipment_id":"S001"}', 'name': 'track_shipment'}, 'type': 'function'}], 'sender': 'Logistics Agent'}
{'role': 'tool', 'tool_call_id': 'call_aNDjzfrUuet6x5fw417ghlX8', 'tool_name': 'track_shipment', 'content': 'Shipment S001 Status:\nCurrent Status: In Transit\nCurrent Location: Atlanta\nEstimated Delivery: 2024-11-08\nCarrier: FedEx'}
{'content': 'The shipment with ID S001 is currently in transit. It is located in Atlanta, and the est

In [43]:

# from swarm.repl import run_demo_loop

# if __name__ == "__main__":
#     run_demo_loop(triage_agent)